In [1]:
cd ..

C:\Users\pi96doc\Documents\Programming\PythonScripts\StateModeling


In [2]:
# %matplotlib notebook
# %matplotlib inline
#import mpld3
#mpld3.enable_notebook()

import StateModeling as stm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from Corona.LoadData import loadData, preprocessData
from Corona.CoronaModel import CoronaModel, plotTotalCases
from bokeh.io import push_notebook, show, output_notebook

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
# import cufflinks as cf

output_notebook()

xpos = np.arange(0,100)/100.0

AllMeasured = loadData(r"COVID-19 Linelist 2020_04_27.xlsx", useThuringia = True, pullData=False)
# AllMeasured = preprocessData(AllMeasured)
# AllMeasured = loadData(useThuringia = False, pullData=False)
ExampleRegions = ['SK Jena', 'LK Greiz'] # 'SK Gera',
AllMeasured = preprocessData(AllMeasured, ReduceDistricts=ExampleRegions, SumDistricts=False, SumAges=True, SumGender=True)

M = CoronaModel(AllMeasured)

if False:
    mobdat = AllMeasured['mobility']
    mobdate = mobdat['date'].to_numpy()
    plt.figure('Retail and recreation');plt.plot(mobdat['retail_and_recreation_percent_change_from_baseline'].to_numpy())
    offsetDay=0; plt.xticks(range(offsetDay, len(mobdate), 7), [date for date in mobdate[offsetDay:-1:7]], rotation="vertical")
    plt.ylabel('Percent Change'); plt.tight_layout()

Tmax = 120

M.toFit(['r0', 'h', 'aT0', 'aBase', 'I0', 'd', 'rd', 'T0', 'q']) # 'q',
if AllMeasured['Cases'].shape[-2] > 1:
    M.toFit.append(['Age Border', 'Age Sigma'])

PopSum = np.sum(AllMeasured['Population'])
measured = AllMeasured['Cases'][:,  np.newaxis, :, :, :] / PopSum
measuredDead = AllMeasured['Dead'][:, np.newaxis, :, :, :] / PopSum
xlim = None  # (60,100)
# fittedVars, fittedRes = M.fit({'detected': measured}, Tmax, otype=otype, oparam=oparam, NIter=NIter, verbose=True, lossScale=lossScale)
FitDict = {'cases': measured}
if "Hospitalized" in AllMeasured.keys():
    FitDict['hospitalization'] = AllMeasured['Hospitalized'][:, np.newaxis, :, :, :]/ PopSum
FitDict['deaths'] = measuredDead

# SimDict = {'cases': None, 'cumul_cases': None, 'cumul_dead':None}

if True:
    otype = "L-BFGS"
    lossScale = 1.0  # 1e4
    oparam = {"normFac": 'max'}
else:
    lossScale = None
    otype = "nesterov"  # "adagrad"  "adadelta" "SGD" "nesterov"  "adam"
    learnrate = {"nesterov": 1000.0, "adam": 7e-7}
    oparam = {"learning_rate": tf.constant(learnrate[otype], dtype=stm.CalcFloatStr)}
# oparam['noiseModel'] = 'Poisson'
oparam['noiseModel'] = 'Gaussian'
# oparam['noiseModel'] = 'ScaledGaussian'  # is buggy? Why the NaNs?

# tf.config.experimental_run_functions_eagerly(True)

def doFit(NIter=0):
    fittedVars, fittedRes = M.fit(FitDict, Tmax, otype=otype, oparam=oparam, NIter=NIter, verbose=True, lossScale=lossScale)

doFit()
# simulated = M.simulate('simulated', FitDict, Tmax=Tmax)

#def showSimRes(ymin=0.0001,ymax=1.0):
#    ylim = [ymin,ymax]
#    M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='occupancy', 
#                  xlim=xlim, dims=("District"), Dates=AllMeasured['Dates'], 
#                  legendPlacement='upper right', styles=['.','-','--'], ylim=ylim, figsize=[10,5])
#    plt.tight_layout()
#    plt.show()
def showSimRes(ymin=0.0001,ymax=1.0):
    doFit()
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='fraction', 
                  xlim=xlim, dims=("District"), subPlot='cases',
                  legendPlacement='upper right',figsize=[10,5])
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='fraction', 
                  xlim=xlim, dims=("District"), subPlot='hospitalization',
                  legendPlacement='upper right',figsize=[10,5])
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='fraction', 
                  xlim=xlim, dims=("District"), subPlot='deaths', 
                  legendPlacement='upper right',figsize=[10,5])
    return p
    

def showSimStates(ymin=1.0,ymax=1.0):
    ylim = [-ymin,ymax]
    M.showStates(MinusOne=('S'), dims2d=None, Dates = AllMeasured['Dates'], ylim=ylim, figsize=[10,5])
    plt.tight_layout()
    plt.show()
    #M.assignNewVar('r0',r0)
    #M.assignNewVar('h',h)
    #simulated = M.simulate('simulated', FitDict, Tmax=Tmax)

def assignParam(d=0.06):
    aDict = {'d':d}
    for varN, newval in aDict.items():
        M.assignNewVar(varN, newval)
        print(varN+": "+str(newval))
    doFit()
    print('Loss: '+str(M.FitResultVars['Loss'].numpy()))



Loading BokehJS ...

unknown age.0... skipping ...
unknown age.0... skipping ...
unknown age.0... skipping ...
unknown age.0... skipping ...
unknown age.0... skipping ...
unknown age.0... skipping ...
tracing doBuildModel
tracing traceModel
tracing time step 119
 .. done

Loss is: tf.Tensor(5.815714e-05, shape=(), dtype=float32)


In [4]:
M.DataDict={}
g = M.getGUI(showResults=showSimRes, doFit=doFit)

HBox()

TraitError: The 'value' trait of a Checkbox instance must be a boolean, but a value of {'name': 'value', 'old': False, 'new': True, 'owner': Checkbox(value=True, description='cumul.', indent=False, layout=Layout(display='flex', width='30%')), 'type': 'change'} <class 'traitlets.utils.bunch.Bunch'> was specified.

In [28]:
#interact_manual(showSim,
#        ymin=ymin,
#        ymax=ymax)

d = widgets.FloatLogSlider(0.06,min=-10,max=2.0,continuous_update=False)
r = widgets.FloatLogSlider(0.01,min=-10,max=2.0,continuous_update=False)
uiS = widgets.HBox((d,r))
allSimWidgets = {'d':d}
print('Simulation Control:')
    
d.observe(assignParam, names='value')
# d.observe(showSimRes, names='value')
# outS = widgets.interactive_output(assignParam, allSimWidgets)
display(uiS, outS)

ymin = widgets.FloatLogSlider(0.001,min=-10,max=3,continuous_update=False)
ymax = widgets.FloatLogSlider(30.0,min=-10,max=3,continuous_update=False)
ui = widgets.HBox((ymin,ymax))
allWidgets = {'ymin': ymin, 'ymax': ymax}
out = widgets.interactive_output(showSimRes, allWidgets)

# out.layout.width = '700px';out.layout.height = '350px'
display(ui, out)

ymin = widgets.FloatLogSlider(0.0001,min=-10,max=6.0,continuous_update=False)
ymax = widgets.FloatLogSlider(0.01,min=-10,max=2.0,continuous_update=False)
ui = widgets.HBox((ymin,ymax))
allWidgets = {'ymin': ymin, 'ymax': ymax}
out2 = widgets.interactive_output(showSimStates, allWidgets)
#out2.layout.width = '700px';out2.layout.height = '350px'
#display(ui, out2)


Simulation Control:


Output(outputs=({'output_type': 'stream', 'text': 'd: 0.039810717055349734\nLoss is: tf.Tensor(0.000104670384,…

Output()

AttributeError: dtype

In [9]:
%matplotlib notebook

In [8]:
from ipywidgets import widgets, Layout
from IPython.display import display
item_layout = Layout(
    display='flex',flex_flow='row',
    justify_content='space-between'
)
box_layout=Layout(
    display='flex',flex_flow='column',
    border='solid 2px',align_items='stretch',width='50%')
tickLayout = Layout(display='flex', width='30%')
inFitWidget = widgets.Checkbox(value=True, indent=False, layout=tickLayout, description='Country')
drop = widgets.Dropdown(options=['a','b'], indent=False, value='a')
dropWidget = widgets.HBox((inFitWidget, drop), display='flex', layout = item_layout)
valueWidget = widgets.FloatLogSlider(value=1.0,base=10,min=-7,max=1)
boxWidget = widgets.HBox((Label('Hi'),valueWidget),layout=item_layout)
# valueWidget = widgets.HBox((inFitWidget,valueWidget))
widget = widgets.Box((dropWidget, boxWidget), layout=box_layout)
display(widgets.HBox((widget,widget,widget, widget)))

In [15]:
valueWidget.description

''

In [4]:
def showSimRes(ymin=0.0001,ymax=1.0):
    doFit()
    p=M.showResultsBokeh(title=AllMeasured['Region'], Scale=PopSum, ylabel='fraction', 
                  xlim=xlim, dims=("District"),
                  legendPlacement='upper right',figsize=[10,5])
    return p


In [5]:
g['T0'].children[1].value

0

In [18]:
M.Var['T0']()

<tf.Tensor: shape=(1, 2, 1, 1), dtype=float32, numpy=
array([[[[2.5118864e-06]],

        [[7.1999998e+00]]]], dtype=float32)>

In [10]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)
p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5),
           background_fill_color='#efefef')
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)
p.xaxis.axis_label = 'Hi'
show(p, notebook_handle=True)